# Transfer Learning
- VGG 모델을 이용해 전이학습 실시
- CIFAR10에 대해 진행

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

In [ ]:
train_batch = 128
test_batch = 16
EPOCH = 200
learning_rate = 0.0001

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.CIFAR10(root='../data', train = True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='../data', train = False,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = train_batch,
                                          shuffle = True, num_workers = 1)

testloader = torch.utils.data.DataLoader(testset, batch_size = test_batch,
                                          shuffle = True, num_workers = 1)

In [ ]:
example = iter(trainloader).next()
ex_image, _ = example

ex_im = ex_image[:8]
ex_im = torchvision.utils.make_grid(ex_im)
ex_im = ex_im.permute(1,2,0)

plt.figure(figsize = (10,10))
plt.imshow(ex_im)

In [ ]:
# model load

from torchvision import models
vgg = models.vgg16(pretrained=True)
vgg

In [ ]:
# 모델 고정하기(학습x)
for p in vgg.features.parameters():
    p.requires_grad = False

In [ ]:
# 출력층 수정하기
vgg.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)

In [ ]:
optimizer = optim.SGD(vgg.classifier.parameters(), lr = learning_rate, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
vgg.to(device)
print(device)

In [ ]:
vgg.train()
import time

for e in range(1, EPOCH+1):
    start_time = time.time()
    running_loss = 0
    for i, data in enumerate(trainloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = vgg(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        
        now = time.time()
        print('\r EPOCH : [%d/%d], ITER : [%d/%d] LOSS : %.7f, Time : %d' 
              %(e, EPOCH, i, len(trainloader), running_loss, now - start_time), end = '')
        
    print('\n')

In [ ]:
vgg.eval()
correct = 0
with torch.no_grad():
    for i, data in enumerate(testloader):
        val_images, val_labels = data
        
        val_images, val_labels = val_images.to(device), val_labels.to(device)
        val_outputs = vgg(val_images)
        pred = val_outputs.argmax(dim=1, keepdim=True)
        correct += pred.eq(val_labels.view_as(pred)).sum().item()
        
print('Accuracy of the network on 10000 test images : %.3f %%' %(100 * correct / len(testloader.dataset)))               

In [ ]:
outputs.shape

In [ ]:
labels

In [ ]:
torch.cuda.empty_cache()